In [ ]:
%pip install -U sentence-transformers
%pip install -U langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [12]:
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer

In [13]:
class Chunk(BaseModel):
    title: str
    content: str
    keywords: list[str]
    named_entities: list[str]
    timestamp_range: str

class ChunkResponse(BaseModel):
    chunks: list[Chunk]

In [14]:
class Chunker:
  def __init__(
    self,
    embed_model: str = "all-MiniLM-L6-v2",
    max_tokens: int = 2_000,
    similarity_threshold: float = 0.82,
    coherence_threshold: float = 0.75,):

    self.embedder = SentenceTransformer(embed_model)
    self.max_tokens = max_tokens
    self.sim_threshold = similarity_threshold
    self.coh_threshold = coherence_threshold

  def TranscriptTextMaker(self, file_path):
    try:
      with open(file_path, "r", encoding="utf-8") as f:
        transcript = f.read()
      print(f"Transcript loaded ({len(self.transcript)} characters)")
    except FileNotFoundError:
        print("Error: transcript.txt file not found")
    except Exception as e:
        print(f"Error loading transcript: {str(e)}")

    return transcript

  def CreatePrompt(self):
    return ChatPromptTemplate.from_template("""
    **SYSTEM PROMPT**
    You are a transcript processing expert. The following transcript needs to be chunked very ingelligently and logically. Ensure sensible segments and structure to be later provided as context to answer questions.

    **INSTRUCTIONS**
    1. Create as many or as few chunks as needed
    2. Each chunk should contain consecutive sentences
    3. For each chunk provide:
      - title: 2-5 word summary
      - content: exact sentences
      - keywords: 3-5 important terms
      - named_entities: any mentioned names
      - timestamp_range: estimate like "00:00-01:30"

    **TRANSCRIPT**
    {input_text}

    **OUTPUT FORMAT**
    {{
      "chunks": [
        {{
          "title": "Summary",
          "content": "Actual sentences",
          "keywords": ["term1", "term2"],
          "named_entities": ["Name"],
          "timestamp_range": "00:00-01:30"
        }}
      ]
    }}
    """)

  def ChunkTranscript(self, llm, file_path, output_result : bool = False) -> list[Chunk]:
    transcript = self.TranscriptTextMaker(file_path)
    try:
      prompt = self.CreatePrompt()
      structured_llm = llm.with_structured_output(ChunkResponse)
      chain = prompt | structured_llm
      response = chain.invoke({"input_text": self.transcript})

      print(f"Generated {len(response.chunks)} chunks")
      if (output_result):
        for i, chunk in enumerate(response.chunks, 1):
          print(f"  Chunk {i}: {chunk.title} ({chunk.timestamp_range}), {chunk.content}")

    except Exception as e:
      print(f"structured output failed: {str(e)}")

    return response.chunks


